In [20]:
import os
import numpy as np
import pandas as pd
import itertools
import warnings
import sys

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
warnings.filterwarnings('ignore')

In [21]:
# Getting the current working directory (cwd)
current_dir = os.getcwd()

# Moving one directory back
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))

# Inserting the path to sys.path
sys.path.insert(1, parent_dir)

from metrics.metrics import rmse, sMAPE, MAE


In [22]:
X_test = pd.read_csv('X_test.csv')
X_train = pd.read_csv('X_train.csv')
y_test = pd.read_csv('y_test.csv')
y_train = pd.read_csv('y_train.csv')
X_train.drop(['day','hour'], axis = 1, inplace = True)
X_test.drop(['day','hour'], axis = 1, inplace = True)
X_train.head()

KeyError: "['day', 'hour'] not found in axis"

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
num_layers = 3
min_nodes_per_layer, max_nodes_per_layer = 128, 512
node_step_size = 64
node_options = list(range(
    min_nodes_per_layer, 
    max_nodes_per_layer + 1, 
    node_step_size
))
layer_possibilities = [node_options] * num_layers

layer_node_permutations = list(itertools.product(*layer_possibilities))
len(layer_node_permutations)

343

In [24]:
import itertools
import torch
import torch.nn as nn

class CustomModel(nn.Module):
    def __init__(self, layers, name):
        super(CustomModel, self).__init__()
        self.network = nn.Sequential(*layers)
        self.name = name  

    def forward(self, x):
        return self.network(x)

def get_models(num_layers: int,
               min_nodes_per_layer: int,
               max_nodes_per_layer: int,
               node_step_size: int,
               input_shape: tuple,
               hidden_layer_activation: str = 'relu',
               num_nodes_at_output: int = 1) -> list:
    
    node_options = list(range(min_nodes_per_layer, max_nodes_per_layer + 1, node_step_size))
    layer_possibilities = [node_options] * num_layers
    layer_node_permutations = list(itertools.product(*layer_possibilities))
    
    activation_functions = {
        'relu': nn.ReLU(),
        'tanh': nn.Tanh(),
        'softmax': nn.Softmax(dim=1)
    }

    models = []
    for permutation in layer_node_permutations:
        layers = []
        input_size = input_shape[0]

        for nodes_at_layer in permutation:
            layers.append(nn.Linear(input_size, nodes_at_layer))
            layers.append(activation_functions[hidden_layer_activation])
            input_size = nodes_at_layer

        layers.append(nn.Linear(input_size, num_nodes_at_output))

        model_name = 'dense' + '_'.join(map(str, permutation)) + '_'
        model = CustomModel(layers, model_name[:-1])  # Create an instance of CustomModel
        models.append(model)
        
    return models




SyntaxError: keyword argument repeated: max_nodes_per_layer (1422640192.py, line 55)

In [25]:
all_models = get_models(
    num_layers=3, 
    min_nodes_per_layer=128,  
    max_nodes_per_layer=448,
    node_step_size=64, 
    input_shape=(6,)
)

NameError: name 'optimization_results' is not defined

# DNN Training with OPTUNA Tuning

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import optuna

In [45]:
import optuna
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler


class RMSELoss(nn.Module):
    def __init__(self):
        super(RMSELoss, self).__init__()
        self.mse = nn.MSELoss()
    
    def forward(self, yhat, y):
        return torch.sqrt(self.mse(yhat, y))



# Load data
X_train = pd.read_csv('X_train.csv')
y_train = pd.read_csv('y_train.csv')
X_test = pd.read_csv('X_test.csv')
y_test = pd.read_csv('y_test.csv')

# Convert DataFrame to numpy array
X_train_np = X_train.to_numpy()
y_train_np = y_train.to_numpy()

# Convert numpy array to PyTorch tensor
X_train_tensor = torch.tensor(X_train_np, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_np, dtype=torch.float32).unsqueeze(1)

# Create dataset and dataloaders
dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=1500, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1500, shuffle=True)

class ElectricityPriceModel(nn.Module):
    def __init__(self, input_dim, layers, units, trial):
        super(ElectricityPriceModel, self).__init__()
        layers_list = []
        layers_list.append(nn.Linear(input_dim, units[0]))
        layers_list.append(nn.ReLU())
        for i in range(1, layers):
            layers_list.append(nn.Linear(units[i-1], units[i]))
            layers_list.append(nn.ReLU())
            p = trial.suggest_float("dropout_l{}".format(i), 0.2, 0.5)
            layers_list.append(nn.Dropout(p))
        layers_list.append(nn.Linear(units[-1], 24))
        self.model = nn.Sequential(*layers_list)

    def forward(self, x):
        return self.model(x)


def objective(trial):
    # Define hyperparameters
    input_dim = X_train.shape[1]
    layers = trial.suggest_int('layers', 1, 3)
    units = [trial.suggest_int(f'units_{i}', 32, 256) for i in range(layers)]

    # Create model
    model = ElectricityPriceModel(input_dim, layers, units, trial).to(device)
    criterion = RMSELoss()
    optimizer = optim.Adam(model.parameters(), lr=trial.suggest_float('lr', 1e-4, 1e-2))

    # Training loop
    num_epochs =  trial.suggest_int('epochs', 20, 50)
    for epoch in range(num_epochs):
        model.train()
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            val_loss += loss.item()

    return val_loss / len(val_loader)

# Initialize device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Create and run the study
study = optuna.create_study(
    direction='minimize',
    #storage="sqlite:///db.sqlite3",
    #study_name="test11"
    )
study.optimize(objective, n_trials=25)

# Print best trial
print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2024-07-17 09:17:20,650] A new study created in memory with name: no-name-e5f26e74-8495-4800-89d5-19ae09579a2c
[I 2024-07-17 09:18:17,149] Trial 0 finished with value: 71.69845581054688 and parameters: {'layers': 3, 'units_0': 212, 'units_1': 196, 'units_2': 166, 'dropout_l1': 0.22674788330322632, 'dropout_l2': 0.41912500648386364, 'lr': 0.0060855911312615975, 'epochs': 21}. Best is trial 0 with value: 71.69845581054688.
[I 2024-07-17 09:20:11,865] Trial 1 finished with value: 73.1899642944336 and parameters: {'layers': 2, 'units_0': 108, 'units_1': 179, 'dropout_l1': 0.46178909360025067, 'lr': 0.00825849384043386, 'epochs': 43}. Best is trial 0 with value: 71.69845581054688.
[I 2024-07-17 09:22:11,159] Trial 2 finished with value: 69.53717193603515 and parameters: {'layers': 2, 'units_0': 68, 'units_1': 242, 'dropout_l1': 0.22557739492965542, 'lr': 0.0008143040465038901, 'epochs': 35}. Best is trial 2 with value: 69.53717193603515.
[I 2024-07-17 09:23:32,009] Trial 3 finished with 

KeyboardInterrupt: 

In [31]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import optuna

# Load data
X_train = pd.read_csv('X_train.csv')
y_train = pd.read_csv('y_train.csv')
X_test = pd.read_csv('X_test.csv')
y_test = pd.read_csv('y_test.csv')

# Ensure all data is numeric
X_train = X_train.apply(pd.to_numeric, errors='coerce')
y_train = y_train.apply(pd.to_numeric, errors='coerce')
X_test = X_test.apply(pd.to_numeric, errors='coerce')
y_test = y_test.apply(pd.to_numeric, errors='coerce')

# Handle missing values (e.g., fill with mean)
X_train = X_train.fillna(X_train.mean())
y_train = y_train.fillna(y_train.mean())
X_test = X_test.fillna(X_test.mean())
y_test = y_test.fillna(y_test.mean())

# Convert DataFrame to numpy array
X_train_np = X_train.to_numpy()
y_train_np = y_train.to_numpy()

# Convert numpy array to PyTorch tensor
X_train_tensor = torch.tensor(X_train_np, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_np, dtype=torch.float32).unsqueeze(1)

# Create dataset and dataloaders
dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Define the model
class ElectricityPriceModel(nn.Module):
    def __init__(self, input_dim, layers, units):
        super(ElectricityPriceModel, self).__init__()
        layers_list = []
        layers_list.append(nn.Linear(input_dim, units[0]))
        layers_list.append(nn.ReLU())
        for i in range(1, layers):
            layers_list.append(nn.Linear(units[i-1], units[i]))
            layers_list.append(nn.ReLU())
        layers_list.append(nn.Linear(units[-1], 1))
        self.model = nn.Sequential(*layers_list)

    def forward(self, x):
        return self.model(x)

# Define the objective function for hyperparameter tuning
def objective(trial):
    # Define hyperparameters
    input_dim = X_train.shape[1]
    layers = trial.suggest_int('layers', 1, 5)
    units = [trial.suggest_int(f'units_{i}', 32, 256, step=32) for i in range(layers)]

    # Create model
    model = ElectricityPriceModel(input_dim, layers, units).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=trial.suggest_loguniform('lr', 1e-4, 1e-2))

    # Training loop
    num_epochs = 50
    for epoch in range(num_epochs):
        model.train()
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            val_loss += loss.item()

    return val_loss / len(val_loader)

# Initialize device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Create and run the study
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)

# Print best trial
print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2024-07-16 17:29:11,073] A new study created in memory with name: no-name-59b93dd3-a106-4584-a295-ca3417b71fba
[W 2024-07-16 17:42:08,680] Trial 0 failed with parameters: {'layers': 3, 'units_0': 160, 'units_1': 256, 'units_2': 192, 'lr': 0.00019191553042241543} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\ubeda\anaconda3\envs\market_forecast\Lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\ubeda\AppData\Local\Temp\ipykernel_28596\3049472072.py", line 90, in objective
    outputs = model(X_batch)
              ^^^^^
  File "C:\Users\ubeda\AppData\Roaming\Python\Python312\site-packages\debugpy\_vendored\pydevd\_pydevd_bundle\pydevd_frame.py", line 988, in trace_dispatch
    self.do_wait_suspend(thread, frame, event, arg)
  File "C:\Users\ubeda\AppData\Roaming\Python\Python312\site-packages\debugpy\_vendored\pydevd\_pydev

KeyboardInterrupt: 